Code Referenced From https://github.com/yasumori

In [ ]:
!pip install python-terrier

import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 3.6 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 303 kB 65.5 MB/s 
     |████████████████████████████████| 46 kB 4.7 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 294 kB 29.1 MB/s 
     |████████████████████████████████| 126 kB 22.9 MB/s 
     |████████████████████████████████| 1.2 MB 52.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 285 kB 97.7 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104091 sha256=b895fde65f300162abe7c4e7de4f5ead7b10c073817fc

In [ ]:
import pyterrier as pt
pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
name_dir=("/content/drive/MyDrive/IRDM")

Mounted at /content/drive


In [ ]:
index_ref = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/ind_3/data.properties")
index= pt.IndexFactory.of(index_ref)

The template and function are for post processing

In [ ]:
id_template = 'spotify:episode:{}_{}'
def get_ep(a,b):
  return id_template.format(a,b) 
import pandas as pd

POS TAGGING (Nouns in topics) with PL2 

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns.xml",format="trecxml")

pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])





res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

PL 2 
{'map': 0.1646699793070074, 'ndcg': 0.419270013930858, 'ndcg_cut_30': 0.2644263695590928, 'P_10': 0.275}


Nouns with BM25 + DPH 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns.xml",format="trecxml")


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(index, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


combined = bm25 + dph 

res=combined.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("DCU Approach")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


DCU Approach
{'map': 0.1899557702382274, 'ndcg': 0.4452750136056026, 'ndcg_cut_30': 0.27400675209389597, 'P_10': 0.3374999999999999}


Other Linear Combinations (DPH, PL2 and BM25) on Nouns

In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.1768791786800509, 'ndcg': 0.43184624515128345, 'ndcg_cut_30': 0.27112592738774033, 'P_10': 0.3125}
pl2 + DPH 
{'map': 0.1656313853097353, 'ndcg': 0.4237350232405159, 'ndcg_cut_30': 0.26392803618423094, 'P_10': 0.275}


QE Approach (Best Performing Approach ) on Nouns

In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:21:30.301 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.17440671662192933, 'ndcg': 0.42271493843418184, 'ndcg_cut_30': 0.26936368933047417, 'P_10': 0.3}


Entities on Topics + Description on PL2

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

PL 2 
{'map': 0.1493216771704032, 'ndcg': 0.3844520020405561, 'ndcg_cut_30': 0.23520555281501193, 'P_10': 0.25}


Entities on Topics + Description on DCU approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(index, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


combined = bm25 + dph 

res=combined.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("DCU Approach")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


DCU Approach
{'map': 0.17610555622448715, 'ndcg': 0.4177186993166014, 'ndcg_cut_30': 0.2501800528171108, 'P_10': 0.2625}


Entities on Topics + Description on Linear Combinations

In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.1677451259801503, 'ndcg': 0.4154473711832167, 'ndcg_cut_30': 0.2486505281844029, 'P_10': 0.2625}
pl2 + DPH 
{'map': 0.15068870897475908, 'ndcg': 0.39529835713260403, 'ndcg_cut_30': 0.2343969973090611, 'P_10': 0.25}


Entities on Topics + Description on QE Approach  (Our approach)

In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DPH QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:35:29.688 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
DPH QE 
{'map': 0.17750900827021465, 'ndcg': 0.39024570386384133, 'ndcg_cut_30': 0.24856738572971832, 'P_10': 0.28750000000000003}


Nouns + Entities on Topics + Description on PL2




In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")

pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

PL 2 
{'map': 0.16588236521052296, 'ndcg': 0.4287557226690556, 'ndcg_cut_30': 0.2637884879143669, 'P_10': 0.275}


Nouns + Entities on Topics + Description on DCU Approach




In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(index, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


combined = bm25 + dph 

res=combined.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("DCU Approach")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


DCU Approach
{'map': 0.19802980927172356, 'ndcg': 0.4513034016532414, 'ndcg_cut_30': 0.2907901220161729, 'P_10': 0.3625}


Nouns + Entities on Topics + Description on Linear Combinations




In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.1811215955248267, 'ndcg': 0.43439482643667493, 'ndcg_cut_30': 0.28348686319061817, 'P_10': 0.325}
pl2 + DPH 
{'map': 0.16869076737438068, 'ndcg': 0.43465317639622963, 'ndcg_cut_30': 0.26473234152257946, 'P_10': 0.28750000000000003}


Nouns + Entities on Topics + Description on QE approach (Our approach)

In [ ]:

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM25+PL2 QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:59:32.756 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
BM25+PL2 QE 
{'map': 0.20147971588703406, 'ndcg': 0.4315828164855843, 'ndcg_cut_30': 0.2850036598688787, 'P_10': 0.35}


Nouns on Test topics QE Approach

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns_test_2.xml",format="trecxml")

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})




import pandas as pd

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("QE BM25+PL2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

14:17:47.139 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE BM25+PL2 
{'map': 0.3226206462745758, 'ndcg': 0.549864429604345, 'ndcg_cut_30': 0.3806985987868705, 'P_10': 0.44791666666666674}


Nouns on Test topics BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns_test_2.xml",format="trecxml")


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(index, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


combined = bm25 + dph 

res=combined.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("DCU Approach")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


DCU Approach
{'map': 0.2882486024424547, 'ndcg': 0.5324078298678572, 'ndcg_cut_30': 0.35981465137176033, 'P_10': 0.44166666666666665}


Nouns on Test topics PL2

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns_test_2.xml",format="trecxml")

pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

PL 2 
{'map': 0.2496039089398373, 'ndcg': 0.4910747877346389, 'ndcg_cut_30': 0.31773733439910223, 'P_10': 0.39166666666666666}


Nouns on Test topics Linear Combinations

In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns_test_2.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.2808048776912048, 'ndcg': 0.5264384399002774, 'ndcg_cut_30': 0.3524291202316769, 'P_10': 0.43124999999999997}
pl2 + DPH 
{'map': 0.25401955873895504, 'ndcg': 0.502952424578475, 'ndcg_cut_30': 0.3218429323468661, 'P_10': 0.39166666666666666}


Entities on Test topics PL2

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

PL 2 
{'map': 0.3412250742253425, 'ndcg': 0.5694572963534915, 'ndcg_cut_30': 0.41006235614097647, 'P_10': 0.4937500000000001}


Entities on Test topics DCU Approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(index, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


combined = bm25 + dph 

res=combined.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("DCU Approach")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


DCU Approach
{'map': 0.3400495145455171, 'ndcg': 0.5701210917975766, 'ndcg_cut_30': 0.4001228859940739, 'P_10': 0.4854166666666668}


Entities on Test topics QE approach 

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})




import pandas as pd

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("Qe ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:49:57.338 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
Qe 
{'map': 0.36030288372643104, 'ndcg': 0.5794349022351173, 'ndcg_cut_30': 0.4225895615765222, 'P_10': 0.5104166666666669}


Entities on Test topics Linear Combinations

In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.3451331814415975, 'ndcg': 0.5748776004691046, 'ndcg_cut_30': 0.4124837168703032, 'P_10': 0.5020833333333335}
pl2 + DPH 
{'map': 0.33836542026941974, 'ndcg': 0.5693647989443454, 'ndcg_cut_30': 0.40230140920518426, 'P_10': 0.48125}


Nouns + Entities on Test topics PL2 

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



import pandas as pd

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

PL 2 
{'map': 0.25054312355139485, 'ndcg': 0.4943986322009808, 'ndcg_cut_30': 0.32195789559831783, 'P_10': 0.39375}


Entities on Test topics BM25 +  DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(index, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(index) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


combined = bm25 + dph 

res=combined.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("DCU Approach")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


DCU Approach
{'map': 0.2898803384726501, 'ndcg': 0.5365128210402591, 'ndcg_cut_30': 0.3652135534757588, 'P_10': 0.4541666666666666}


Nouns + Entities on Test topics QE approach 

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")

qe = pt.BatchRetrieve(index, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(index, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(index, controls={'wmodel': "PL2","c":20})




import pandas as pd

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)


print("Qe ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

00:53:25.603 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
Qe 
{'map': 0.3227009562267826, 'ndcg': 0.5511497833569156, 'ndcg_cut_30': 0.38245059216522725, 'P_10': 0.45208333333333345}


Nouns + Entities on Linear Combinations

In [ ]:
bm25 = pt.BatchRetrieve(index, controls={"wmodel":"BM25","c":0.6}, metadata=["docno",'text'],properties={"bm25.k_1":0.5})
ret= pt.BatchRetrieve(index, controls={"wmodel": "DPH"})
pl2= pt.BatchRetrieve(index, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")

mod= bm25+pl2
mod2= pl2+dph
import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=mod.transform(topics)
res_2=mod2.transform(topics)
df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df_2=pd.DataFrame(res_2, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
df_2["ep_id"]=df_2["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df_2["episode"]=df_2["docno"].apply(lambda x: x.split("_")[0])
df_2["docno"]=df_2.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df_2["docno"]=df_2["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
rsl_2=df_2[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("BM 25 + PL2")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))
print("pl2 + DPH ")
print(pt.Utils.evaluate(rsl_2,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

BM 25 + PL2
{'map': 0.28206831342035726, 'ndcg': 0.5301961316031978, 'ndcg_cut_30': 0.35821210001536535, 'P_10': 0.44166666666666665}
pl2 + DPH 
{'map': 0.25524317920162215, 'ndcg': 0.5059805883504825, 'ndcg_cut_30': 0.32666473522597866, 'P_10': 0.3958333333333333}
